In [1]:
# find a schedule that minimizes the total amount of time
# or cost required to complete all the tasks.
# Employee Scheduling
from ortools.constraint_solver import pywrapcp

In [2]:
# create the solver
solver = pywrapcp.Solver('schedule_shifts')

In [3]:
# config
num_nurses = 4
num_shifts = 4  # nurse assigned to shift 0 means not working that day.
num_days = 7

In [4]:
# create shift variables.
# shift to nurse.
shifts = {}
for j in range(num_nurses):
    for i in range(num_days):
        shifts[(j, i)] = solver.IntVar(0, num_shifts-1, "shifts(%i, %i)" % (j, i))
shifts_flat = [shifts[(j, i)] for j in range(num_nurses) for i in range(num_days)]

In [5]:
# create nurse variables.
# nurse to shift.
nurses = {}
for j in range(num_shifts):
    for i in range(num_days):
        nurses[(j, i)] = solver.IntVar(0, num_nurses-1, "shift%d day%d" % (j, i))

In [6]:
# define the relationship between nurses and shifts
for day in range(num_days):
    nurses_for_day = [nurses[(j, day)] for j in range(num_shifts)]
    for i in range(num_nurses):
        s = shifts[(i, day)]
        solver.Add(s.IndexOf(nurses_for_day) == i)

In [7]:
# nurses work different shifts
# Make assignments different on each day
for i in range(num_days):
    solver.Add(solver.AllDifferent([shifts[(j, i)] for j in range(num_nurses)]))
    solver.Add(solver.AllDifferent([nurses[(j, i)] for j in range(num_shifts)]))

In [8]:
# Nurses work five or six days per week
# each nurse works 5 or 6 days in a week.
for j in range(num_nurses):
    solver.Add(solver.Sum([shifts[(j, i)] > 0 for i in range(num_days)]) >= 5)
    solver.Add(solver.Sum([shifts[(j, i)] > 0 for i in range(num_days)]) <= 6)

In [9]:
# shifts are staffed by at most two nurses per week.
works_shift = {}
# works_shift[(j, i)] is 1 if nurse i works shift j at least one day in the week.
for i in range(num_nurses):
    for j in range(num_shifts):
        works_shift[(i, j)] = solver.BoolVar("shift%d nurse%d" % (i, j))

for i in range(num_nurses):
    for j in range(num_shifts):
        solver.Add(works_shift[(i, j)] == solver.Max([shifts[i, k] == j for k in range(num_days)]))

In [10]:
for j in range(1, num_shifts):
    solver.Add(solver.Sum([works_shift[(i, j)] for i in range(num_nurses)]) <= 2)

In [11]:
# nurses work shift 2 or 3 on consecutive days
solver.Add(solver.Max(nurses[(2, 0)] == nurses[(2, 1)], nurses[(2, 1)] == nurses[(2, 2)]) == 1)
solver.Add(solver.Max(nurses[(2, 1)] == nurses[(2, 2)], nurses[(2, 2)] == nurses[(2, 3)]) == 1)
solver.Add(solver.Max(nurses[(2, 2)] == nurses[(2, 3)], nurses[(2, 3)] == nurses[(2, 4)]) == 1)
solver.Add(solver.Max(nurses[(2, 3)] == nurses[(2, 4)], nurses[(2, 4)] == nurses[(2, 5)]) == 1)
solver.Add(solver.Max(nurses[(2, 4)] == nurses[(2, 5)], nurses[(2, 5)] == nurses[(2, 6)]) == 1)
solver.Add(solver.Max(nurses[(2, 5)] == nurses[(2, 6)], nurses[(2, 6)] == nurses[(2, 0)]) == 1)
solver.Add(solver.Max(nurses[(2, 6)] == nurses[(2, 0)], nurses[(2, 0)] == nurses[(2, 1)]) == 1)

solver.Add(solver.Max(nurses[(3, 0)] == nurses[(3, 1)], nurses[(3, 1)] == nurses[(3, 2)]) == 1)
solver.Add(solver.Max(nurses[(3, 1)] == nurses[(3, 2)], nurses[(3, 2)] == nurses[(3, 3)]) == 1)
solver.Add(solver.Max(nurses[(3, 2)] == nurses[(3, 3)], nurses[(3, 3)] == nurses[(3, 4)]) == 1)
solver.Add(solver.Max(nurses[(3, 3)] == nurses[(3, 4)], nurses[(3, 4)] == nurses[(3, 5)]) == 1)
solver.Add(solver.Max(nurses[(3, 4)] == nurses[(3, 5)], nurses[(3, 5)] == nurses[(3, 6)]) == 1)
solver.Add(solver.Max(nurses[(3, 5)] == nurses[(3, 6)], nurses[(3, 6)] == nurses[(3, 0)]) == 1)
solver.Add(solver.Max(nurses[(3, 6)] == nurses[(3, 0)], nurses[(3, 0)] == nurses[(3, 1)]) == 1)

In [12]:
# Add the decision builder and solution collector
# create the decision builder.
db = solver.Phase(shifts_flat, solver.CHOOSE_FIRST_UNBOUND,
                 solver.ASSIGN_MIN_VALUE)
# create the solution collector.
solution = solver.Assignment()
solution.Add(shifts_flat)
collector = solver.AllSolutionCollector(solution)

In [13]:
# call the solver and display the results
solver.Solve(db, [collector])
print("Solutions found:", collector.SolutionCount())
print("Time: ", solver.WallTime(), "ms")
print()

# display a few solutions picked at random.
a_few_solutions = [859, 2034, 5091, 7003]
for sol in a_few_solutions:
    print("Solution number", sol, '\n')
    
    for i in range(num_days):
        print("Day", i)
        for j in range(num_nurses):
            print('Nurse', j, "assigned to task", collector.Value(sol, shifts[(j, i)]))
        print()

Solutions found: 18144
Time:  3436792 ms

Solution number 859 

Day  0
Nurse 0 assigned to task 0
Nurse 1 assigned to task 2
Nurse 2 assigned to task 3
Nurse 3 assigned to task 1

Day  1
Nurse 0 assigned to task 0
Nurse 1 assigned to task 2
Nurse 2 assigned to task 3
Nurse 3 assigned to task 1

Day  2
Nurse 0 assigned to task 2
Nurse 1 assigned to task 1
Nurse 2 assigned to task 3
Nurse 3 assigned to task 0

Day  3
Nurse 0 assigned to task 2
Nurse 1 assigned to task 0
Nurse 2 assigned to task 3
Nurse 3 assigned to task 1

Day  4
Nurse 0 assigned to task 2
Nurse 1 assigned to task 0
Nurse 2 assigned to task 3
Nurse 3 assigned to task 1

Day  5
Nurse 0 assigned to task 3
Nurse 1 assigned to task 2
Nurse 2 assigned to task 0
Nurse 3 assigned to task 1

Day  6
Nurse 0 assigned to task 3
Nurse 1 assigned to task 2
Nurse 2 assigned to task 0
Nurse 3 assigned to task 1

Solution number 2034 

Day  0
Nurse 0 assigned to task 0
Nurse 1 assigned to task 1
Nurse 2 assigned to task 3
Nurse 3 assig